In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 

In [3]:
data = pd.read_csv("./SPUG_tiny_data.csv",index_col=0)
data.head()

,9611,100132707,574494,54622,440465,64151,441282,5524,23344,57178,...,102682016,390038,4784,9577,54453,9439,554223,441272,103344929,8823
GSM1404124,0.593278,0.159713,0.552773,0.887201,0.752376,0.064782,0.577760,0.459988,0.075172,0.098989,...,0.261808,0.581119,0.363503,0.791346,0.112213,0.791812,0.884257,0.773579,0.688271,0.805395
GSM1438264,0.671212,0.503595,0.668371,0.671141,0.350839,0.106301,0.478525,0.172701,0.122911,0.106552,...,0.597478,0.753756,0.572624,0.740464,0.195841,0.793637,0.732863,0.850680,0.813435,0.729730
GSM1269760,0.754687,0.481456,0.642046,0.529795,0.749447,0.082483,0.613443,0.332416,0.091326,0.167530,...,0.095672,0.777432,0.556090,0.809779,0.250353,0.775489,0.802736,0.870004,0.671667,0.657290
GSM886067,0.824935,0.442685,0.677556,0.538332,0.781661,0.055583,0.631201,0.278238,0.082505,0.092253,...,0.543296,0.582827,0.448941,0.811930,0.255680,0.743877,0.767874,0.826049,0.235668,0.811777
GSM1404009,0.638180,0.226426,0.547676,0.818637,0.704749,0.062868,0.541751,0.322037,0.085700,0.102336,...,0.627477,0.544007,0.397251,0.826884,0.119193,0.710059,0.850233,0.773076,0.761416,0.382550
